In [1]:
%load_ext cython
import sys
sys.path.append('/home/bram/ANTS')
sys.path.append('/home/bram/ANTS/entropy')
for p in sys.path:
    print(p)


/usr/lib/python36.zip
/usr/lib/python3.6
/usr/lib/python3.6/lib-dynload
/home/bram/PythonEnvs/Ants/lib/python3.6/site-packages
/home/bram/PythonEnvs/Ants/lib/python3.6/site-packages/IPython/extensions
/home/bram/.ipython
/home/bram/ANTS
/home/bram/ANTS/entropy


In [4]:
%%cython
from cythonic.core.queen cimport queen
from cythonic.core.domain cimport domain
from cythonic.core.ant cimport Ant as ant
from entropy.core.visualization import StigmergyPlot, Plotter
import numpy as np
cimport numpy as cnp
cimport cython

cdef double pitch = .5
cdef double[2] limits = np.array([1000,1000],dtype=np.float_)
cdef double[2] size = limits, nest = np.array([25.,50.]), food = np.array([75.,50.])
cdef double R=5.
dom_settings = {'size':size, 'pitch': pitch, 'nest_loc': nest, 'food_loc':food,
                'nest_rad':R, 'food_rad': R}
ant_dict = {'speed' : 10, 'gain' : 2, 'l' : 5, 'sens_offset' : 45,
          'limits' :np.array(limits),'q' : 1.5, 'return_factor' :1,
          'drop_fun' : 'exp_decay', 'drop_beta' : .5, 'rng_gamma':2}

@cython.cdivision(True)
@cython.wraparound(False)
@cython.boundscheck(False)
@cython.nonecheck(False)
cdef class sim_controller():
    cdef domain dom
    cdef queen controller
    cdef ant solo
    
    def __cinit__(self, dom_dict, ant_dict={}):
        print('init')
        self.dom = domain(**dom_dict)
        self.dom.set_target_pheromone(self.dom.Map.sum())
        self.solo = ant(id =1, **ant_dict)
        
    cpdef cnp.ndarray[ndim = 1,dtype=cnp.npy_float64] get_pos(self):
        return np.array([self.solo._pos.x,self.solo._pos.y],dtype = np.float_)
    
    cdef void evaporate(self):
        self.dom.pyvaporate()
        
    cdef void cevaporate(self):
        self.dom.cvaporate()
        
cdef sim_controller sim = sim_controller(dom_dict = dom_settings, ant_dict = ant_dict)
print(sim.get_pos())

from time import time




sim.solo.init_positions(np.array([500,500],dtype=np.float_))
sim.dom.init_gaussian(significancy=1e3, sigma=3)
# vis = Plotter(sim.dom.Map, shown='stigmergy')

print(time())
cdef double toc,tic, t_npy,t_c
cdef unsigned int i,N =20
tic = time()
for i in range(N):
    sim.evaporate()
toc = time()
t_npy = toc-tic
print(f"Evaporate took {t_npy*1000/N} msec")
tic = time()
for i in range(N):
    sim.cevaporate()
toc = time()
t_c=toc-tic
print(f"C-Evaporate took {t_c*1000/N} msec ")
print(np.array(sim.dom.Map.map).sum())
print(f"C is {t_npy/t_c} times faster")
print('--done--')

init
[0. 0.]
1543491889.042821
Evaporate took 105.94269037246704 msec
C-Evaporate took 14.214789867401123 msec 
4004001.0
C is 7.4529902559745995 times faster
--done--
